In [1]:
from typing import Dict, Union, Sequence, Set, Optional, cast, List
import logging

from overrides import overrides
from conllu import parse_incr
from itertools import chain

from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import Field, TextField, SequenceLabelField, MetadataField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer

In [ ]:
words = ['The', 'team', 'who', 'work', 'there', 'are', 'helpfull', ',', 'friendly', 'and', 'extremely', 'knowledgeable', 'and', 'will', 'help', 'you', 'as', 'much', 'as', 'they', 'can', 'with', 'thier', 'years', 'of', 'hands', 'on', 'practice', '.']
pos = ['DET','NOUN','PRON','VERB','ADV','AUX','ADJ','PUNCT''ADJ','CCONJ','ADV','ADJ','CCONJ,''AUX','VERB','PRON','ADV','ADV','SCONJ','PRON','AUX','ADP','PRON','NOUN','ADP','NOUN''ADV','NOUN','PUNCT']
head_ids = [2, 7, 4, 2, 4, 7, 0, 9, 7, 12, 12, 7, 15, 15, 7, 15, 18, 15, 21, 21, 17, 24, 24, 15, 28, 28, 26, 24, 7]
deps = [[('det', 2)], [('nsubj', 4), ('nsubj', 7), ('nsubj', 9), ('nsubj', 12), ('nsubj', 15)], [('ref', 2)], [('acl:relcl', 2)], [('advmod', 4)], [('cop', 7)], [('root', 0)], [('punct', 9)], [('conj:and', 7)], [('cc', 12)], [('advmod', 12)], [('conj:and', 7)], [('cc', 15)], [('aux', 15)], [('conj:and', 7)], [('obj', 15)], [('advmod', 18)], [('advmod', 15)], [('mark', 21)], [('nsubj', 21)], [('advcl:as', 17)], [('case', 24)], [('nmod:poss', 24)], [('obl:with', 15)], [('case', 28)], [('compound', 28)], [('compound', 26)], [('nmod:of', 24)], [('punct', 7)]]

In [ ]:
words = ['The', 'situation', 'in', 'Iraq', 'is', 'only', 'going', 'to', 'get', 'better', 'this', 'way', '.']
tags =  ['det', 'nsubj', 'case', 'nmod', 'aux', 'advmod', 'root', 'mark', 'xcomp', 'xcomp', 'det', 'obj', 'punct']
heads = [2, 7, 4, 2, 7, 7, 0, 9, 7, 9, 12, 10, 7]
deps = [[('det', 2)], [('nsubj', 7), ('nsubj:xsubj', 9), ('nsubj:xsubj', 10)], [('case', 4)], [('nmod:in', 2)], [('aux', 7)], [('advmod', 7)], [('root', 0)], [('mark', 9)], '7:xcomp _', '9:xcomp _', [('det', 12)], [('obj', 10)], '7:punct _']

In [ ]:
words = ['Over', '300', 'Iraqis', 'are', 'reported', 'dead', 'and', '500', 'reported', 'wounded', 'in', 'Fallujah', 'alone', '.']
pos = ['ADV', 'NUM', 'PROPN', 'AUX', 'VERB', 'ADJ', 'CCONJ', 'NUM', 'VERB', 'ADJ', 'ADP', 'PROPN', 'ADV','PUNCT']
head_tags = [['advmod'], ['nummod'], ['nsubj:pass', 'nsubj:xsubj', 'nsubj:pass'], ['aux:pass'], ['root'], ['xcomp'], ['cc', 'cc'], ['conj:and', 'nsubj:pass', 'nsubj:xsubj'], ['conj:and'], ['xcomp'], ['case'], ['obl:in'], ['advmod'], ['punct']]
head_indices = [[2], [3], [5, 6, 8], [5], [0], [5], [8, (8, '.', 1)], [5, (8, '.', 1), 9], [5], [(8, '.', 1)], [11], [5], [11], [5]]

In [2]:
words = ['Over', '300', 'Iraqis', 'are', 'reported', 'dead', 'and', '500', 'reported', 'wounded', 'in', 'Fallujah', 'alone', '.']
pos_tags = ['ADV', 'NUM', 'PROPN', 'AUX', 'VERB', 'ADJ', 'CCONJ', 'NUM', 'VERB', 'ADJ', 'ADP', 'PROPN', 'ADV', 'PUNCT']
dependencies = [('advmod', 2), ('nummod', 3), ('nsubj:pass', 5), ('aux:pass', 5), ('root', 0), ('xcomp', 5), ('cc', 8), ('conj', 5), ('_', None), ('orphan', 8), ('case', 11), ('obl', 5), ('advmod', 11), ('punct', 5)]
deps = [[('advmod', 2)], [('nummod', 3)], [('nsubj:pass', 5), ('nsubj:xsubj', 6), ('nsubj:pass', 8)], [('aux:pass', 5)], [('root', 0)], [('xcomp', 5)], [('cc', 8), ('cc', (8, '.', 1))], [('conj:and', 5), ('nsubj:pass', (8, '.', 1)), ('nsubj:xsubj', 9)], [('conj:and', 5)], [('xcomp', (8, '.', 1))], [('case', 11)], [('obl:in', 5)], [('advmod', 11)], [('punct', 5)]]


In [7]:
orig = dependencies
#[('det', 2), ('nsubj', 7), ('nsubj', 4), ('acl:relcl', 2), ('advmod', 4), ('cop', 7), ('root', 0), ('punct', 9), ('conj', 7), ('cc', 12), ('advmod', 12), ('conj', 7), ('cc', 15), ('aux', 15), ('conj', 7), ('obj', 15), ('advmod', 18), ('advmod', 15), ('mark', 21), ('nsubj', 21), ('advcl', 17), ('case', 24), ('nmod:poss', 24), ('obl', 15), ('case', 28), ('compound', 28), ('compound', 26), ('nmod', 24), ('punct', 7)]

#deps
#[[('det', 2)], [('nsubj', 4), ('nsubj', 7), ('nsubj', 9), ('nsubj', 12), ('nsubj', 15)], [('ref', 2)], [('acl:relcl', 2)], [('advmod', 4)], [('cop', 7)], [('root', 0)], [('punct', 9)], [('conj:and', 7)], [('cc', 12)], [('advmod', 12)], [('conj:and', 7)], [('cc', 15)], [('aux', 15)], [('conj:and', 7)], [('obj', 15)], [('advmod', 18)], [('advmod', 15)], [('mark', 21)], [('nsubj', 21)], [('advcl:as', 17)], [('case', 24)], [('nmod:poss', 24)], [('obl:with', 15)], [('case', 28)], [('compound', 28)], [('compound', 26)], [('nmod:of', 24)], [('punct', 7)]]

In [5]:
orig

[('advmod', 2),
 ('nummod', 3),
 ('nsubj:pass', 5),
 ('aux:pass', 5),
 ('root', 0),
 ('xcomp', 5),
 ('cc', 8),
 ('conj', 5),
 ('_', None),
 ('orphan', 8),
 ('case', 11),
 ('obl', 5),
 ('advmod', 11),
 ('punct', 5)]

In [6]:
deps

[[('advmod', 2)],
 [('nummod', 3)],
 [('nsubj:pass', 5), ('nsubj:xsubj', 6), ('nsubj:pass', 8)],
 [('aux:pass', 5)],
 [('root', 0)],
 [('xcomp', 5)],
 [('cc', 8), ('cc', (8, '.', 1))],
 [('conj:and', 5), ('nsubj:pass', (8, '.', 1)), ('nsubj:xsubj', 9)],
 [('conj:and', 5)],
 [('xcomp', (8, '.', 1))],
 [('case', 11)],
 [('obl:in', 5)],
 [('advmod', 11)],
 [('punct', 5)]]

In [65]:
##1 NORMAL

heads = []
rels = []

for target_output in deps:
    # check if there is just 1 head
    if len(target_output) == 1:
        head = [x[1] for x in target_output]
        rel = [x[0] for x in target_output]
        heads.append(head)
        rels.append(rel)
    # more than 1 head
    else:
        # append all current target heads/rels to a list
        current_heads = []
        current_rels = []
        for rel_head_tup in target_output:
            current_heads.append(rel_head_tup[1])
            current_rels.append(rel_head_tup[0])
        heads.append(current_heads)
        rels.append(current_rels)

In [66]:
heads

[[2],
 [3],
 [5, 6, 8],
 [5],
 [0],
 [5],
 [8, (8, '.', 1)],
 [5, (8, '.', 1), 9],
 [5],
 [(8, '.', 1)],
 [11],
 [5],
 [11],
 [5]]

In [67]:
rels

[['advmod'],
 ['nummod'],
 ['nsubj:pass', 'nsubj:xsubj', 'nsubj:pass'],
 ['aux:pass'],
 ['root'],
 ['xcomp'],
 ['cc', 'cc'],
 ['conj:and', 'nsubj:pass', 'nsubj:xsubj'],
 ['conj:and'],
 ['xcomp'],
 ['case'],
 ['obl:in'],
 ['advmod'],
 ['punct']]

In [44]:
heads

[[2], [3], [5], [0], [5], [5], [(8, '.', 1)], [11], [5], [11], [5]]

In [14]:
rels

[['advmod'],
 ['nummod'],
 ['aux:pass'],
 ['root'],
 ['xcomp'],
 ['conj:and'],
 ['xcomp'],
 ['case'],
 ['obl:in'],
 ['advmod'],
 ['punct']]

In [122]:
# process the elided tokens
processed_heads = []

for head_list in heads:
    # keep a list-of-lists format
    current_heads = []
    for head in head_list:
        # convert copy node tuples: (8, '.', 1) to float: 8.1
        if type(head) == tuple:
            copy_node = list(head)
            # join the values in the tuple
            copy_node = str(head[0]) + '.' + str(head[-1])   
            copy_node = float(copy_node)
            current_heads.append(copy_node)
        else:
            # regular head index
            current_heads.append(head)

    processed_heads.append(current_heads)


In [123]:
heads

[[2],
 [3],
 [5, 6, 8],
 [5],
 [0],
 [5],
 [8, (8, '.', 1)],
 [5, (8, '.', 1), 9],
 [5],
 [(8, '.', 1)],
 [11],
 [5],
 [11],
 [5]]

In [125]:
processed_heads

[[2],
 [3],
 [5, 6, 8],
 [5],
 [0],
 [5],
 [8, 8.1],
 [5, 8.1, 9],
 [5],
 [8.1],
 [11],
 [5],
 [11],
 [5]]

In [126]:
print(len(words))
print(len(heads))
print(len(processed_heads))

14
14
14


In [121]:
assert len(words) == len(heads) == len(procesed_heads)

In [ ]:
##1 NORMAL

heads = []
rels = []

for target_output in deps:
    # check if there is just 1 head
    if len(target_output) == 1:
        head = [x[1] for x in target_output]
        rel = [x[0] for x in target_output]
        heads.append(head)
        rels.append(rel)
    # more than 1 head
    else:
        # append all current target heads/rels to a list
        current_heads = []
        current_rels = []
        for rel_head_tup in target_output:
            current_heads.append(rel_head_tup[1])
            current_rels.append(rel_head_tup[0])
        heads.append(current_heads)
        rels.append(current_rels)

In [ ]:
##1

heads = []
rels = []

for target_output in deps:
    #print(target_output)
    # check if there is just 1 head
    if len(target_output) == 1:
        #head = [x[1] for x in target_output]
        # 0 = tuple, 1 is head (2nd elem of tuple)
        head = target_output[0][1]
        #print(head)
    #    rel = [x[0] for x in target_output]
        heads.append(head)
    #    rels.append(rel)
    # more than 1 head
    else:
        # append all current target heads/rels to a list
        current_heads = []
    #    current_rels = []
        print(target_output)
        for rel_head_tup in target_output:
            #print(rel_head_tup)

            current_heads.append(rel_head_tup[1])
    #        current_rels.append(rel_head_tup[0])
        hz = "+".join(str(h) for h in current_heads)
        heads.append(hz)
    #    rels.append(current_rels)

In [ ]:
heads
#rels

In [ ]:
##2
#labels : ``Union[List[str], List[int]]``

heads = []
rels = []

for target_output in deps:
    # check if there is just 1 head
    if len(target_output) == 1:
        head = [x[1] for x in target_output]
        rel = [x[0] for x in target_output]
        heads.append(head)
        rels.append(rel)
    # more than 1 head
    else:
        # append all current target heads/rels to a list
        current_heads = []
        current_rels = []
        for rel_head_tup in target_output:
            current_heads.append(rel_head_tup[1])
            current_rels.append(rel_head_tup[0])
        heads.append(current_heads)
        rels.append(current_rels)

In [ ]:
deps = list(zip(head_tags, head_indices))

In [ ]:
# maybe add a copy embedding? (bad at test time)
for i, w in enumerate(words):
    print(w, "==>", deps[i])

In [ ]:
head_indices

In [ ]:
for x in head_indices:
    print(x)

In [ ]:
print(head_indices)

In [ ]:
# process the elided tokens
for x in head_indices:
    #print(x)
    #print(type(x)) # - all lists
    for y in x:
        #print(y)
        #print(type(y))
        if type(y) == tuple:
            z = list(y)
            z = str(y[0]) + '.' + str(y[-1])
            y = z
            #print(head_indices[y])
            #head_indices[y] = z
            #head
                

In [ ]:
print(head_indices[0])

In [ ]:
head_ids = list(chain.from_iterable(heads))
head_ids

In [ ]:
a = [x for x in heads]
a

In [ ]:
targs = list(zip(heads, rels))
targs

In [ ]:
yo = [x[0] for x in heads]
yo

In [ ]:
for i, w in enumerate(words):
    print(w, ",", pos[i], "==>", head_ids[i])

In [ ]:
for i, w in enumerate(words):
    print(w, ",", pos[i], "==>", heads[i])

In [ ]:
print(heads)

In [ ]:
w_i = "team"
h_i = [4, 7, 9, 12, 15]
r_i = ['nsubj', 'nsubj', 'nsubj', 'nsubj', 'nsubj']

In [ ]:
pred = list(zip(h_i, r_i))
pred

In [ ]:
heads

In [ ]:
print(w_i, pred)

In [ ]:
List[List]

In [ ]:
fields: Dict[str, Field] = {}
fields

In [ ]:
## See : https://allenai.github.io/allennlp-docs/api/data/fields/multilabel_field/
# https://github.com/allenai/allennlp/pull/1054
# https://github.com/allenai/allennlp/pull/1054/commits/8f576a74d9ecf5fc817a0ca9c9f303fc400e58d6
# https://github.com/allenai/allennlp/blob/master/allennlp/data/fields/multilabel_field.py

In [ ]:
# fields pos {pos: a, b c d}
# fields heads {heads: [a, b, c], [a], [b]}

In [ ]:
d['a'] = 100  # existing key, so overwrite
d['c'] = 3  # new key, so add
d['d'] = 4

In [ ]:
d = {}

In [ ]:
a = [[1,2,3],[3,2,1],[6, 6, 6]]

In [ ]:
for letter in a:
    #print(letter)
    d["loss"] = a

In [ ]:
d

In [ ]:
x ={}

In [ ]:
x["loss"]